In [3]:
file = open("C:/Users/PRADEEP REDDY/Desktop/IIITH-Summer Internship/Telugu-Question-Answering-master/Telugu-Question-Answering-master/Data/Questions/date.txt", "rb")
all_ques = []
for q in file:
    q = q.decode('utf-8').split('\n')[0]
    all_ques.append(q)
file.close()
#print(all_ques)
#print(len(all_ques))
################## Answers for the above questions ############
import os
for qu_in in range(len(all_ques)):
    if(os.path.exists('web.txt')):
        os.remove('web.txt')
    ###################################  Question Classification  #########################################
    input_query = all_ques[qu_in] #input("Enter your Telugu Query for Question_Classification: ")
    classes = []
    questions = []
    label_generalization = []
    ans_type = ''
    f = open("Data_for_Question_classification.txt" , "rb")
    for line in f:
        #print(line.decode('utf-8'))
        line = line.decode('utf-8').strip('\n')
        #print(line.split()[0])
        label = (line.split()[0]).split(":")[0]    # accessing the labels for the questions in the data.
        classes.append(label)
        questions.append(line[6:])
        #print(line[6:])
        if(label=="PERS"):
            label_generalization.append(1)         # Generalizing the classes
        elif(label=="LOCA"):
            label_generalization.append(2)
        elif(label=="ORGA"):
            label_generalization.append(3)
        elif(label=="DATE"):
            label_generalization.append(4)
        elif(label=="TIME"):
            label_generalization.append(5)
        elif(label=="PERC"):
            label_generalization.append(6)
        elif(label=="NUMB"):
            label_generalization.append(7)
    #print(classes)
    #print(len(classes))
    #print(questions)
    #print(len(questions))
    #print(label_generalization)
    #print(len(label_generalization))
    
    ################## Training and Testing phase #################
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.neural_network import MLPClassifier
    from sklearn.svm import LinearSVC
    from sklearn.linear_model import LogisticRegression
    from sklearn import metrics
    
    test_query = input_query
    vectorizer = TfidfVectorizer()
    questions.append(test_query)
    X = vectorizer.fit_transform(questions)
    #print(vectorizer.get_feature_names())
    #print(X.shape)
    
    X1 = X[:len(label_generalization)]
    Y1 = label_generalization[:len(label_generalization)]
    X_train, X_test, y_train, y_test = train_test_split(X1, Y1, train_size = 0.8, test_size = 0.2, random_state = 48)
    #print(X_test)
    #print(y_test)
    
    classifier_LogisticRegression = LogisticRegression(random_state=0,multi_class="ovr",solver="lbfgs")
    #print(classifier_LogisticRegression)
    
    classifier_LogisticRegression.fit(X_train,y_train)
    
    test1_query = X[-1].toarray()
    #print(X[-1])   # returns the doument_id, token_id, Tfidf score
    #print(test1_query)
    
    
    prediction3 = classifier_LogisticRegression.predict(X_test)
    
    print("Accuracy: ", metrics.accuracy_score(y_test,prediction3)*100)
        
    ########################### Predicting the User Input query Class ##################################
    print('\n')
    ip_query_predict = classifier_LogisticRegression.predict(test1_query)
    if(ip_query_predict == 1):
        predicted_output_query = "PERSON"
    elif(ip_query_predict == 2):
        predicted_output_query = "LOCATION"
    elif(ip_query_predict == 3):
        predicted_output_query = "ORGANIZATION"
    elif(ip_query_predict == 4):
        predicted_output_query = "DATE"
    elif(ip_query_predict == 5):
        predicted_output_query = "TIME"
    elif(ip_query_predict == 6):
        predicted_output_query = "PERCENTAGE"
    else:
        predicted_output_query = "NUMBER"
            
    #print("\n%s belongs to the class [%s]" %(test_query, predicted_output_query))
        
    ans_type += predicted_output_query


            
            
    #################################### Translation of Telugu query to English ################################
    from googletrans import Translator
    trans = Translator()
    en = trans.translate(input_query,dest = 'en',src = 'te')
    tel_to_eng = en.text
    
    print(str(qu_in+1)+": "+"Telugu to English: ", tel_to_eng)
    
    
    ####################################  Getting the access to the web  #####################################
    
    import requests, webbrowser
    from bs4 import BeautifulSoup
    import re
    
    query1 = tel_to_eng   #input("Enter your search query: ")
    query = query1.lower()
    search_engine1 = 'google' #input("Enter the search engine to search, \"Google\" or \"Bing\": ")
    search_engine = search_engine1.lower()
    no_of_urls = 5 #int(input("Enter the number of search results You required: "))
    
    url_list=[]
    
    def access_web(search_engine,query,url_list):
        res = requests.get('https://' + search_engine + '.com/search?q=' + query)
        soup = BeautifulSoup(res.text, 'lxml')
        
        if (search_engine == 'google'):                    ####### Using Google search engine #######
            search_results = soup.select('div#main > div > div > div > a')
            for i in range(len(search_results)):
                url_list.append(search_results[i].get('href'))
                #webbrowser.open('https://' + search_engine + '.com' + search_results[i].get('href'))
            #print(url_list)
            
        elif (search_engine == 'bing'):                   ####### Using Bing search engine ########
            if not(soup.findAll('li', attrs={'class':'b_algo'})):
                access_web(search_engine,query,url_list)
            for i in soup.find_all('li', class_ = 'b_algo'):
                j = i.find('h2')
                for url in j.find_all('a', href=True):
                    url_list.append(url['href'])
                    #webbrowser.open('https://' + search_engine +'.com' + url['href'])
            #print (url_list)
        else:
            print("Enter the search engines either 'Google' or 'Bing' only......")
        return url_list
    
    url_list = access_web(search_engine,query,url_list)
    
    ########################## Getting the Content from the web ##########################
    Sentences=[]
    url_list1 = url_list[:no_of_urls]
    for i in range(no_of_urls):
        try:
            if (search_engine == 'google'):
                res1 = requests.get('https://' + search_engine + '.com' + url_list1[i])
                res1.raise_for_status()
            else:
                res1 = requests.get(url_list1[i])
                res1.raise_for_status()
            content = BeautifulSoup(res1.text,'lxml')
            
            
            if (i==0):
                f = open('web.txt', 'wb')
                for para in content.findAll(['p']):
                    paragraph = para.text
                    Sentences.append(paragraph.strip())
                    f.write(bytes(paragraph,'utf-8'))
                f.close()
            else:
                f1 = open('web.txt', 'ab')
                for para in content.findAll(['p']):
                    paragraph = para.text
                    Sentences.append(paragraph.strip())
                    f1.write(bytes(paragraph,'utf-8'))
                f1.close()
        except requests.HTTPError as e:
            print (e)
        except IOError as e:
            print(e)
        except ValueError as e:
            print(e)
        except Exception as e:
            print(e)
            #print(" ")
            #raise
    #print(Sentences)
    
    
    
    ############################    Finding the similarity matrix  ##########################
    
    import nltk
    from nltk.tokenize import word_tokenize,sent_tokenize
    a = Sentences
    universal_sentence = ""
    all_tokens = []
    for i in range(len(a)):
        universal_sentence += (a[i] + ' ')
        
    One_sentence = sent_tokenize(universal_sentence)
    for j in One_sentence:
        all_tokens.append(word_tokenize(j))
        
    all_tokens_combined = word_tokenize(str.lower(universal_sentence))
    all_unique_tokens_combined = list(set(all_tokens_combined))
    
    
    ###################################   Integer Encoding of all sentences  ###########################################
    
    #print(all_unique_tokens_combined,"\n\n")
    integer_encoding = {}
    for i,word in enumerate(all_unique_tokens_combined):
        
        integer_encoding[word] = i
    #print("\n\n------------------------------------Integer Encoding---------------------------------------\n\n")
    #print("Integer Encoded words:\n",integer_encoding)
    
    ################################  Integer encoding for each sentence  ########################################
    
    def integer_encoding1(subset_sentence):
        subset_sentence_tokens = word_tokenize(subset_sentence)
        subset_sentence_tokens_encoded = {}
        for i in subset_sentence_tokens:
            j = i
            if (j not in integer_encoding.keys()):
                j+='.'
            if (j not in integer_encoding.keys()):
                integer_encoding[j] = len(integer_encoding)
            subset_sentence_tokens_encoded[j] = integer_encoding[j]
            #subset_sentence_tokens_encoded.append(integer_encoding[i])
        return (subset_sentence_tokens_encoded)
    
    #################################################  Cosine Similarity    #####################################################
    
    import math
    def Cosine_Similarity(vec1, vec2):
        common = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in common])
        
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = (math.sqrt(sum1) * math.sqrt(sum2))
        if not denominator:
            return(0.0)
        
        else:
            return(float(numerator) / denominator)
        
    text1 = query
    ret_sent = {}
    for k in range(len(One_sentence)):
        text2 = str.lower(One_sentence[k])
        vector1 = integer_encoding1(text1)
        vector2 = integer_encoding1(text2)
        cosine_similar = Cosine_Similarity(vector1, vector2)
        ret_sent[One_sentence[k]] = cosine_similar
    
    #print(ret_sent)
    
    
    ####################################### Picking Most relevant sentences ###################################################
    
    print("\n\n\n\n")
    no_of_relevant_sentences = 40 #int(input("Enter the number of relevant sentences you require:"))
    def sort_dict(d):
        return sorted(d.items(), key = lambda x : x[1], reverse = True)
    most_relevant_sentences = sort_dict(ret_sent)
    #print(most_relevant_sentences)
    
    for l in range(min(no_of_relevant_sentences,len(most_relevant_sentences))):
        print(most_relevant_sentences[l][0],'\n')
        
        
    
    ############################  NER Tagging #######################
    print('\n\n\t\t\t\t--------NER Tagging starts------')
    
    import spacy
    nlp = spacy.load('en_core_web_sm')
    
    possible_entities_dictionary = {}
    possible_entities_dictionary_modified = {}
    
    for j in range(min(no_of_relevant_sentences,len(most_relevant_sentences))):
        entities = []
        text = most_relevant_sentences[j][0]
        doc = nlp(text)
        #print(doc)
        
        for entity in doc.ents:
            entities.append([entity.text, entity.label_])
        #print(entities)
        
        for ne, label in entities:
            if (label not in possible_entities_dictionary.keys()):
                possible_entities_dictionary[label] = [ne]
                
            else:
                possible_entities_dictionary[label].append(ne)
        
        for i in possible_entities_dictionary.keys():
            if(i == 'CARDINAL'):
                possible_entities_dictionary_modified['NUMBER'] = possible_entities_dictionary[i]
            elif(i == 'GPE'):
                possible_entities_dictionary_modified['LOCATION'] = possible_entities_dictionary[i]
            elif(i == 'ORG'):
                possible_entities_dictionary_modified['ORGANIZATION'] = possible_entities_dictionary[i]
            elif(i == 'PERCENT'):
                possible_entities_dictionary_modified['PERCENTAGE'] = possible_entities_dictionary[i]
            else:
                if(i in ['PERSON','DATE','TIME']):
                    possible_entities_dictionary_modified[i] = possible_entities_dictionary[i]
                    
                    
    print('\n',possible_entities_dictionary,'\n\n')
    
    print('\n-----------------------------Possible entities dictionary----------------------------\n')
    
    print(possible_entities_dictionary_modified,'\n\n')
    if (ans_type in possible_entities_dictionary_modified.keys()):
        possible_answers = possible_entities_dictionary_modified[ans_type]
    else:
        possible_answers = ['no answer']
    
    print("Possible answers: ", possible_answers,"\n\n")

    ####################### Converting Best answer to Telugu #########################
    
    from googletrans import Translator     # Works only with Internet
    from nltk.tokenize import word_tokenize
    import math
    
    english_query = tel_to_eng
    possible_entities_dictionary1 = possible_entities_dictionary_modified
    
    all_possible_answers1 = possible_answers[:]
    
    token_words = word_tokenize(english_query.lower())
    
    translator = Translator()
    
    #print(token_words)
    
    
    def Cosine_Similarity1(vec1, vec2):
        common = set(vec1.keys()) & set(vec2.keys())
        #print(common)
        numerator = sum([vec1[x] * vec2[x] for x in common])
        
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = (math.sqrt(sum1) * math.sqrt(sum2))
        if not denominator:
            return(0.0)
        else:
            return(float(numerator) / denominator)
        
    text1 = tel_to_eng
    
    for k in range(len(possible_answers)):
        text2 = str.lower(possible_answers[k])
        vector1 = integer_encoding1(text1.lower())
        #print(vector1)
        vector2 = integer_encoding1(text2)
        #print(vector2)
        cosine_similar1 = Cosine_Similarity1(vector1, vector2)
        #print(cosine_similar1)
        if (cosine_similar1 > 0.0):
            all_possible_answers1.remove(possible_answers[k])
            
    print(all_possible_answers1)
    
    all_possible_answers = all_possible_answers1
    
    dict1 = {}
    for i in all_possible_answers:
        if i not in dict1.keys():
            dict1[i] = 1
        else:
            dict1[i] += 1
    #print(dict1)
    if(len(dict1.values())>0):
        max_count = max(dict1.values())
        for k in dict1.keys():
            if (dict1[k] == max_count):
                Best_answer = k
                break
    else:
        Best_answer = 'no answer'
    
    if(Best_answer.isnumeric()):
        Best_answer1 = Best_answer
        print("Ans: ",Best_answer1)
        
    else:
        a = translator.translate(Best_answer, dest='te',src='auto')
        #print(a)
        Best_answer1 = a.text
        print("Ans: ",Best_answer1)
    
    
    que_trans = all_ques[qu_in]
    
    if(qu_in == 0):
        write_content = ans_type+':'+' '+que_trans[:]+"@@@"+Best_answer1+'\n'
        my_file = open("C:/Users/PRADEEP REDDY/Desktop/IIITH-Summer Internship/Open Domain Questoin Answering/Week-6/LR/Date/LR_dat_40_ans_pred.txt","wb")
        my_file.write(bytes(write_content,'utf-8'))
        print(write_content)
    else:
        write_content = ans_type+':'+' '+que_trans[:]+"@@@"+Best_answer1+'\n'
        my_file = open("C:/Users/PRADEEP REDDY/Desktop/IIITH-Summer Internship/Open Domain Questoin Answering/Week-6/LR/Date/LR_dat_40_ans_pred.txt","ab")
        my_file.write(bytes(write_content,'utf-8'))
        print(write_content)
    

Accuracy:  75.46468401486989


1: Telugu to English:  In which year did India conduct its first nuclear weapons test?
HTTPSConnectionPool(host='google.comhttps', port=443): Max retries exceeded with url: //fas.org/nuke/guide/india/nuke/first-pix.htm?sa=X&ved=2ahUKEwi8lca0vaTqAhVwxzgGHbu0Bp0Q9QF6BAgFEAI (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002F084BE8A00>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))





[8] The first nuclear bomb project did not employ more than 75 scientists because of its sensitivity. 

[9] The Indian public had supported the nuclear tests which ultimately led Prime Minister Narasimha Rao deciding to conduct further tests in 1995. 

India started its own nuclear programme in 1944 when Homi Jehangir Bhabha founded the Tata Institute of Fundamental Research. 

"[7] Bhabha did not realise, however, that the U.S. Plowshare cost-figures were produced by a vast industrial complex costing tens of bill

Ans:  పాకిస్థాన్
LOCATION: ఏ సంవత్సరంలో భారతదేశం తన మొదటి అణు ఆయుధం పరీక్ష నిర్వహించారు?@@@పాకిస్థాన్

Accuracy:  75.46468401486989


2: Telugu to English:  In which year did Goa become the 25th state of India?





Gaumkari thus were in existence long before constitution of the state of Goa itself. 

[64] Because of state conflicts with the Jesuits, In 1762 the Marquês de Pombal expelled the order from Macau. 

Goa celebrates Liberation Day on 19 December every year, which is also a state holiday. 

[1] In spite of being India's smallest state by area, Goa's history is both long and diverse. 

The Dutch war did finally end in 1663. 

The Goa Inquisition was the office of the Inquisition acting within the Indian state of Goa and the rest of the Portuguese empire in Asia. 

In 1542, St. Francis Xavier mentions the architectural splendour of the city. 

Pratapsingh Rane, who had previously served as Chief Minister of Goa, Daman and Diu, was elected as the first Chief Minister of the newl

Ans:  ఈరోజు
DATE: ఏ సంవత్సరంలో గోవా భారతదేశం యొక్క 25 వ రాష్ట్రముగా అవతరించినది?@@@ఈరోజు

Accuracy:  75.46468401486989


3: Telugu to English:  In which year did Mahatma Gandhi die?
HTTPSConnectionPool(host='google.comhttps', port=443): Max retries exceeded with url: //starsunfolded.com/mahatma-gandhi/?sa=X&ved=2ahUKEwiqobPDvaTqAhVk4jgGHTzLBVIQ_B16BAgHEAI (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002F0F6264EB0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))





The car in which the couple was travelling was delayed and the first car reached the auditorium. 

Ominous cries of ‘Let Gandhi die!’ were heard in Delhi, where Gandhi was occupying a mansion called Birla Lodge. 

The government of India, led by Congress leaders, had withheld a payment to Pakistan in January 1948 because it did not want to finance Pakistan, which was at war with India at that time. 

[6] According to The Guardian report, which did not mention H

Ans:  జనవరి 1948
DATE: ఏ సంవత్సరంలో  మహాత్మా గాంధీ మరణించారు?@@@జనవరి 1948

Accuracy:  75.46468401486989


4: Telugu to English:  On October 3, Germany celebrated 25 years of the reunification of West Germany and East Germany. But when did West Germany and East Germany reintegrate?
HTTPSConnectionPool(host='google.comhttps', port=443): Max retries exceeded with url: //en.wikipedia.org/wiki/German_reunification?sa=X&ved=2ahUKEwiI7L3KvaTqAhXRyzgGHYYcDtwQ9QF6BAgHEAI (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002F080078E80>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))





It replaced the previous national holiday held in West Germany on 17 June commemorating the Uprising of 1953 in East Germany and the national holiday on 7 October in the GDR, that commemorated the foundation of the East German state. 

When the first Five-Year Plan was announced, the flow of refugees out of East Germany began to grow. 

In the east, mig

Ans:  17 జూన్
DATE: పశ్చిమ జర్మనీ మరియు తూర్పు జర్మనీ పునరేకీకరణ యొక్క 25 సంవత్సరాల ఉత్సవాన్ని  జెర్మనీ అక్టోబర్ 3  న జరుపుకుంది.  అయితే  ఎప్పుడు పశ్చిమ జర్మనీ మరియు తూర్పు జర్మనీ పునరేకీకరణ చెందాయి?@@@17 జూన్

Accuracy:  75.46468401486989


5: Telugu to English:  In which year was the Battle of Plassey?
HTTPSConnectionPool(host='google.comhttps', port=443): Max retries exceeded with url: //en.wikipedia.org/wiki/Battle_of_Plassey?sa=X&ved=2ahUKEwj6vurTvaTqAhUkyDgGHRpjAuIQ_B16BAgQEAI (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002F081EA8220>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))





Seven Years War The Battle of Plassey was fought in north-eastern India on 23 June 1757. 

[3] Tensions and suspicions between Siraj-ud-daulah and the British culminated in the Battle of Plassey. 

Battle of Plassey,  (23 June 1757). 

Though it was more of a skirmish than a battle, the British victory under Robert Clive at Plassey i

...

Accuracy:  75.46468401486989


106: Telugu to English:  Jharkhand became a state in India in which year did it leave Bihar?
HTTPSConnectionPool(host='google.comhttps', port=443): Max retries exceeded with url: //www.britannica.com/place/Jharkhand?sa=X&ved=2ahUKEwj45fGjxKTqAhXyxDgGHS0xCqkQ9QF6BAgNEAI (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002F080CBB430>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))





After the Independence of India, the region became part of Bihar state. 

There was demand for a separate state in the region and the Bihar Reorganisation Act, 2000 passed in Parliament, giving rise to the new state of Jharkhand. 

The tribal groups initiated a call for independence from Bihar, and in the 1980s they became militant in their demand. 

"Bushland" or "The land of forest") is a state in eastern India. 

In 2000 a campaign led by the BJP for a separate state culminated with the passage of the Bihar Reorga

Accuracy:  75.46468401486989


107: Telugu to English:  World Laugh Day is a day to celebrate the world.





This includes the following… How to Celebrate World Laughter Day Want to have a good laugh? 

On World Laughter Day people usually go to Laughter club, gather and laugh together. 

Hence, to mark the importance of laughter, World laughter day is celebrated every year. 

The celebration of World Laughter Day is a positive manifestation for world peace and is intended to build up a global consciousness of brotherhood and friendship through laughter. 

But as we know that the whole world is facing a tough situation due to COVID-19 the celebrations will be held online. 

[2][4] The celebration of World Laughter Day is a positive manifestation for world peace and is intended to build up a global consciousness of brotherhood and friendship through laughter. 

Go to a laughter club. 

Hundreds of people gather worldwide on that day to laugh together. 

Every year more than 70 countrie


 {'PERSON': ['Benazir Bhutto', 'Benazir Bhutto', 'Shahid Aziz', 'Kargil', 'Nawaz Sharif', 'Atal Bihari Vajpayee', 'victory[1][2][3][4][5', 'Pervez Musharraf', 'Pervez Musharraf', 'Bipin Rawat'], 'EVENT': ['the Kargil War', 'The Kargil War', 'the Kargil War', 'the Atlantique Incident', 'the Kargil War', 'The Kargil War', 'the 1999 Cricket World Cup', 'Cricket World Cup', 'the First Kashmir War', 'the Kargil War'], 'GPE': ['Pakistan', 'Kargil', 'India', 'Pakistan', 'Kargil', 'India', 'Pakistan', 'Kargil', 'Pakistan', 'Musharraf', 'Kargil', 'Pakistan', 'Pakistan', 'India', 'Kargil', 'India', 'Pakistan', 'India', 'Pakistan', 'Kargil', 'India', 'Pakistan', 'India', 'Pakistan', 'Kargil', "the People's Republic of Bangladesh", 'Pakistan', 'Kargil', 'Pakistan', 'Kargil', 'India', 'India', 'Pakistan', 'Pakistan', 'Kargil', 'Kargil', 'Jammu', 'Pakistan', 'India', 'Pakistan', 'Kargil', 'Dras', 'Kargil', 'Pakistan', 'Pakistan', 'Jammu', 'India', 'Kargil', 'India', 'Kargil', 'Pakistan', 'Kargil', 


 {'GPE': ['India', 'India', 'India', 'India', 'India', 'India', 'India', 'India', 'India', 'Delhi', 'India', 'Delhi', 'Library', 'India', 'India', 'India', 'India', 'Hyderabad State', 'India', 'India', 'U.S.', 'the Soviet Union', 'Ketan', 'Portugal', 'India', 'India', 'India', 'India', 'India', 'India', 'India', 'India', 'India', 'India', 'India'], 'ORG': ['Jawaharlal Nehru', 'Jawaharlal Nehru', 'Jawaharlal Nehru', 'BJP', 'The Jawaharlal Nehru University', 'the Nehru Memorial Museum', 'PM', 'Jawaharlal Nehru', 'Jawaharlal Nehru', "Mehta's", 'the Indian National Congress', 'the Indian Army', 'Jawaharlal Nehru', 'YbKaym rbqzCqYzloAgljgltYLramyIItdGRcveMupUm British Broadcasting CorporationHome Accessibility', 'the National Book Trust', 'the National Literary Academy', 'Jawaharlal Nehru', 'the Indian National Congress', 'the Congress party', 'Congress', 'Social Reform\r\n\t\t\t\t'], 'ORDINAL': ['first', 'first', 'first', 'first', 'first', 'first', 'first', 'first'], 'PERSON': ['Allahabad


 {'EVENT': ['the Second World War', 'World War\n', 'World War II', 'the Second World War', 'the First World War', 'World War Two', 'World War Two', 'The Second World War', 'World War II', 'World War II', 'World War', 'World War II', 'World War II', 'World War', 'World War Two', 'World War II', 'the Korean War', 'World War II', 'the civil war', 'World War II', 'World War', 'World War I', 'World War', 'World War One'], 'PERSON': ['Chamberlain', 'Hitler', 'Wehrmacht'], 'CARDINAL': ['1.25 million', '400', '70 to 85 million', 'over 30', 'over 50 million', '40,000,000–50,000,000', '59', '316', 'Some 350,000', '13'], 'NORP': ['Germans', 'Soviet', 'Japanese', 'Nationalists', 'British', 'communist', 'German', 'European', 'European'], 'GPE': ['Poland', 'Britain', 'Germany', 'Poland', 'Germany', 'Poland', 'Britain', 'Germany', 'Korea', 'Germany', 'Poland', 'China', 'Germany', 'Norway', 'Denmark', 'Germany', 'the Soviet Union', 'Battle of Britain', 'Austria-Hungary', 'Germany', 'Bulgaria', 'the O